In [2]:
import os
import re
from typing import TypedDict, List, Dict
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv

load_dotenv()

# 🔥 Fast + Stable Model
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.3
)

# ---------------------------------------------------
# 1️⃣ Define Agent State (Updated for Milestone 4)
# ---------------------------------------------------

class AgentState(TypedDict):
    messages: List
    todos: List[str]
    completed_todos: List[str]
    files: Dict[str, str]
    sub_agent_results: Dict[str, str]  # Added for Milestone 3 integration
    current_todo: str
    final_output: str
    evaluation_percentage: float
    evaluation_verdict: str

# ---------------------------------------------------
# 2️⃣ Upgraded SYSTEM PROMPT for Autonomy
# ---------------------------------------------------

SYSTEM_PROMPT = """
You are a Deep Cognitive Autonomous Agent ("The Master Architect").

You solve complex, vague, multi-step tasks using:

1. Strategic Planning
2. Context Storage
3. Sub-Agent Delegation
4. Step-by-Step Execution
5. Final Synthesis

RULES:

- Only analyze elements explicitly mentioned in the user's request.
- Break tasks logically into sub-tasks.
- If a research task requires specialized data, spawn a sub-agent.
- If a summary exceeds 500 words, offload to the file system.
- Maintain structured notes in your virtual file system.
- Always provide a clear, justified conclusion if required.
- Be strictly relevant and context-aware.
"""

# ---------------------------------------------------
# 3️⃣ Planner Node
# ---------------------------------------------------

def planner(state: AgentState):

    user_request = state["messages"][-1].content

    prompt = f"""
{SYSTEM_PROMPT}

Break the following complex task into 4 to 6 structured,
high-level sub-tasks.

User Request:
{user_request}

Return tasks as a clean numbered list only.
"""

    response = llm.invoke([
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=prompt)
    ])

    todos = [
        t.strip("- ").strip()
        for t in response.content.split("\n")
        if t.strip()
    ]

    state["todos"] = todos[:6]
    state["completed_todos"] = []

    return state

# ---------------------------------------------------
# 4️⃣ Orchestrator Node (New) – Conditional Execution
# ---------------------------------------------------

def orchestrator(state: AgentState):

    remaining = [
        t for t in state["todos"]
        if t not in state["completed_todos"]
    ]

    if not remaining:
        return state

    current = remaining[0]
    state["current_todo"] = current

    # Decide if we need a sub-agent (e.g., research, external data)
    if "research" in current.lower() or "data" in current.lower():
        # Spawn Sub-Agent
        result = spawn_sub_agent(current)
        state["sub_agent_results"][current] = result
        state["completed_todos"].append(current)
    else:
        # Use main executor
        state = executor(state)

    return state

# ---------------------------------------------------
# 5️⃣ Executor Node (unchanged)
# ---------------------------------------------------

def executor(state: AgentState):

    current = state["current_todo"]
    user_request = state["messages"][0].content

    prompt = f"""
{SYSTEM_PROMPT}

You are executing this specific sub-task:

{current}

This sub-task is part of this overall objective:

{user_request}

Provide a structured, focused output.
Limit response to 400–600 words.
Stay strictly relevant.
"""

    response = llm.invoke([
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=prompt)
    ])

    content = response.content[:2500]

    filename = current[:30].replace(" ", "_") + ".txt"
    state["files"][filename] = content

    state["completed_todos"].append(current)

    return state

# ---------------------------------------------------
# 6️⃣ Sub-Agent Node (Simulated)
# ---------------------------------------------------

def spawn_sub_agent(task: str) -> str:
    """
    Simulated sub-agent: performs specialized research or data fetch
    """
    prompt = f"""
{SYSTEM_PROMPT}

You are a specialized sub-agent.

Perform deep research for the following task:

{task}

Return concise, structured findings.
"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content[:2500]

# ---------------------------------------------------
# 7️⃣ Synthesizer Node
# ---------------------------------------------------

def synthesizer(state: AgentState):

    user_request = state["messages"][0].content

    compiled_notes = "\n\n".join(
        [f"{k}:\n{v}" for k, v in {**state["files"], **state["sub_agent_results"]}.items()]
    )

    compiled_notes = compiled_notes[:6000]

    prompt = f"""
{SYSTEM_PROMPT}

You are the Final Synthesis Engine.

Original User Request:
{user_request}

Structured Findings:
{compiled_notes}

Produce a clear, well-structured final output.
"""

    response = llm.invoke([
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=prompt)
    ])

    state["final_output"] = response.content

    # Save final report
    state["files"]["FINAL_REPORT.txt"] = response.content

    return state

# ---------------------------------------------------
# 8️⃣ Evaluator Node (LLM-as-Judge)
# ---------------------------------------------------

def evaluator(state: AgentState):

    user_request = state["messages"][0].content
    final_output = state["final_output"]

    judge_prompt = f"""
You are a strict evaluator.

User Request:
{user_request}

Agent Output:
{final_output}

Score 1 to 5 for each category.

Return EXACTLY in this format:

Structure: <number>
Logic: <number>
Constraint: <number>
Depth: <number>
Conclusion: <number>


Only numbers. No explanation.
"""

    response = llm.invoke([HumanMessage(content=judge_prompt)])
    text = response.content

    # 🔥 Robust parsing
    scores = re.findall(r"\b[1-5]\b", text)

    if len(scores) >= 5:
        scores = list(map(int, scores[:5]))
        total = sum(scores)
        percentage = (total / 25) * 100
    else:
        percentage = 0

    if percentage >= 80:
        verdict = "Excellent"
    elif percentage >= 60:
        verdict = "Good"
    elif percentage >= 40:
        verdict = "Average"
    else:
        verdict = "Poor"

    state["evaluation_percentage"] = percentage
    state["evaluation_verdict"] = verdict

    print("\n==============================")
    print(f"Evaluation Score: {percentage:.2f}%")
    print(f"Verdict: {verdict}")
    print("==============================\n")

    return state

# ---------------------------------------------------
# 9️⃣ Build StateGraph Workflow
# ---------------------------------------------------

workflow = StateGraph(AgentState)

workflow.add_node("planner", planner)
workflow.add_node("orchestrator", orchestrator)
workflow.add_node("executor", executor)
workflow.add_node("synthesizer", synthesizer)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "orchestrator")

workflow.add_conditional_edges(
    "orchestrator",
    lambda state: (
        "continue"
        if len(state["completed_todos"]) < len(state["todos"])
        else "done"
    ),
    {
        "continue": "orchestrator",
        "done": "synthesizer"
    }
)

workflow.add_edge("synthesizer", END)

app = workflow.compile()

# ---------------------------------------------------
# 10️⃣ Test
# ---------------------------------------------------

long_input = """
Plan a two-day national-level technical fest for an engineering college.
Include budget planning, marketing, sponsor strategy, scheduling, and risk management.
"""

initial_state = {
    "messages": [HumanMessage(content=long_input)],
    "todos": [],
    "completed_todos": [],
    "files": {},
    "sub_agent_results": {},
    "current_todo": "",
    "final_output": "",
    "evaluation_percentage": 0.0,
    "evaluation_verdict": ""
}

result = app.invoke(initial_state)

# 🔥 Call evaluator manually
result = evaluator(result)

print("\n========== FINAL OUTPUT ==========\n")
print(result["final_output"])
print("\nFinal Score:", result["evaluation_percentage"], "%")
print("Final Verdict:", result["evaluation_verdict"])



Evaluation Score: 92.00%
Verdict: Excellent


========== FINAL OUTPUT ==========

**Final Synthesis: Comprehensive Plan for a Two-Day National-Level Technical Fest**

**Executive Summary:**

The proposed two-day national-level technical fest, "InnovateX," aims to provide a platform for engineering students from top colleges to showcase their innovative ideas, skills, and projects. The event will be held on the college campus, with a budget of ₹5 lakhs (approximately $6,700 USD). The plan includes a comprehensive marketing strategy, sponsor strategy, scheduling, and risk management.

**Event Concept:**

* **Theme:** "Innovate. Evolve. Excel."
* **Target Audience:** Engineering students from top colleges
* **Event Duration:** Two days (9:00 AM - 6:00 PM)
* **Event Venue:** College campus (indoor and outdoor spaces)

**Marketing Strategy:**

* **Social Media:** Utilize social media platforms (Facebook, Twitter, Instagram, LinkedIn) to promote the event, share updates, and engage with pot